In [0]:
dbutils.notebook.run("Delete Experiment", 0)

In [0]:
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/processed/datmarcobre_featureengineer_delta")
df_delta.show()

In [0]:
datos = df_delta.toPandas()
datos.head()

In [0]:
%pip install mlflow hyperopt fsspec

In [0]:
import pandas as pd
import sklearn.model_selection
import sklearn.ensemble

#gf
import mlflow
import mlflow.keras
import mlflow.tensorflow
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from hyperopt.pyll import scope

from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [0]:
datos['numero_pases_carguio'].unique()

In [0]:
# Supongamos que datos es tu DataFrame y has seleccionado tus características (X) y variable objetivo (y)
X = datos[['capacidad_en_volumen_equipo_carguio_m3',
'capacidad_en_peso_equipo_carguio',
'capacidad_en_peso_equipo_acarreo',
'densidad_inicial_poligono_creado_tn/m3',
'capacidad_en_volumen_equipo_acarreo_m3']].values

y = datos['numero_pases_carguio'].values # Reemplaza 'variable_objetivo' con el nombre de tu variable objetivo

In [0]:
datos.dtypes

## Part 1. Train a classification model

### MLflow Tracking
[MLflow tracking](https://www.mlflow.org/docs/latest/tracking.html) allows you to organize your machine learning training code, parameters, and models. 

You can enable automatic MLflow tracking by using [*autologging*](https://www.mlflow.org/docs/latest/tracking.html#automatic-logging).

In [0]:
# Establecer semillas en NumPy y TensorFlow
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

Modelo 1 - Redes LSTM

In [0]:
#Librerias para redes neuronales(LSTM)
from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Establecer semillas en NumPy y TensorFlow
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X_array, y_array, test_size=0.2, random_state=42)

model_rnn = Sequential()
# La forma de entrada para LSTM debe ser (n_timesteps, n_features)
model_rnn.add(LSTM(30, activation='relu', input_shape=(1, 5)))  #  (n_samples, n_pasos, n_variables)
# Agregar capas Dense según sea necesario
model_rnn.add(Dense(60, activation='relu'))
model_rnn.add(Dense(30, activation='relu'))
model_rnn.add(Dense(15, activation='relu'))
# Capa de salida
model_rnn.add(Dense(1, activation='linear'))
#-----------------------------------------------------------------------------------------------------------------------------------------

# Compilar el modelo
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
# Imprimir un resumen del modelo
model_rnn.summary()

# Entrenar el modelo
history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=1, validation_data=(X_test_rnn.reshape(-1, 1, 5),y_test_rnn), batch_size=5, verbose=1)

# VALIDACION 1
# Obtener la pérdida en el conjunto de entrenamiento y el conjunto de validación
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Calcular el RMSE en el conjunto de entrenamiento y el conjunto de validación
train_rmse = np.sqrt(train_loss)
val_rmse = np.sqrt(val_loss)

# VALIDACION 2
# Hacer predicciones en el conjunto de prueba
y_pred_rnn = model_rnn.predict(X_test_rnn.reshape(-1, 1, 5)) # (n_samples, n_pasos, n_variables)

# Redondear los valores de y_pred al entero más cercano
y_pred_rnn = np.round(y_pred_rnn).astype('int64')

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_rnn, y_pred_rnn)
# Calcular la raíz cuadrada del error cuadrático medio
rmse = np.sqrt(mse)
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

In [0]:
import mlflow
import mlflow.keras
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Establecer semillas en NumPy y TensorFlow
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

#MLflow podra registrar automáticamente métricas, parámetros y el modelo generado durante el entrenamiento
mlflow.tensorflow.autolog()  # Keras y TensorFlow

# Entrenar el modelo
with mlflow.start_run(run_name='experimento_MLOPS_V1'):  #Iniciliza un nuevo experimento de MlFlow
    # Definir y compilar el modelo
    model_rnn = Sequential()
    model_rnn.add(LSTM(30, activation='relu', input_shape=(1, 5)))
    model_rnn.add(Dense(60, activation='relu'))
    model_rnn.add(Dense(30, activation='relu'))
    model_rnn.add(Dense(15, activation='relu'))
    model_rnn.add(Dense(1, activation='linear'))

    model_rnn.compile(loss='mean_squared_error', optimizer='adam')

    # Entrenar el modelo y guardar el historial del entrenamiento
    history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=1, validation_data=(X_test_rnn.reshape(-1, 1, 5), y_test_rnn), batch_size=5, verbose=1)

    # Save the run information to register the model later
    #kerasURI = run.info.artifact_uri

    # Registrar los parámetros del modelo
    mlflow.log_params({
        "input_shape": (1, 5),
        "lstm_units": 30,
        "dense_layers": [60, 30, 15],
        "activation": "relu",
        "optimizer": "adam",
        "loss_function": "mean_squared_error",
        "epochs": 1,
        "batch_size": 5
    })

    # Registrar la estructura del modelo en MLflow
    #mlflow.keras.log_model(model_rnn, "keras_tf_v1")

    # VALIDACION 
    # Obtener la pérdida en el conjunto de entrenamiento y el conjunto de validación
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']

    # Calcular el RMSE en el conjunto de entrenamiento y el conjunto de validación
    train_rmse = np.sqrt(train_loss)
    test_rmse = np.sqrt(test_loss)

    # Registro de métricas de RMSE
    mlflow.log_metric("train_RMSE", train_rmse[-1])  # Se registra el último valor de RMSE del conjunto de entrenamiento
    mlflow.log_metric("test_RMSE", test_rmse[-1])      # Se registra el último valor de RMSE del conjunto de validación
    
    # Crear gráfico de pérdida durante el entrenamiento
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
    plt.plot(epochs, test_loss, 'r', label='Pérdida en validación')
    plt.title('Pérdida durante el entrenamiento y la validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.savefig("kerasplotv3.png")
    mlflow.log_artifact("kerasplotv3.png")

In [0]:
model_path

#### Register to Model Registry

In [0]:
model_name = "Example_rnn"                                  #nombre que se le dara al modelo que registraremos en el MlFlow Registry
model_uri = f"runs:/ed91e81d7ca644728f3e223fa3cfac92/model" #identificador unico del experimento a registrar en Mlflow
registered_model_version = mlflow.register_model(model_uri, model_name)

#### Load from Model Registry 

In [0]:
import numpy as np
import mlflow

client = mlflow.tracking.MlflowClient() 
model_name = "Example_rnn"

model_info = client.get_registered_model(model_name)
model_version = model_info.latest_versions[0].version #Ultima version del modelo en MlFlow Registry
#model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# Supongamos que tienes tus datos de entrada en una lista
input_data = np.array([[12, 35, 129, 48, 129]])

# # Convertir los datos de entrada al tipo de datos float64
input_data = input_data.astype(np.float64)

# # Aumentamos a 3dimensiones los datos de entrada para que tengan la forma (None, 1, 5)
input_data = np.reshape(input_data, (input_data.shape[0], 1, input_data.shape[1]))

# # Realiza la predicción utilizando el modelo
model.predict(input_data)

### Load model without experiments

In [0]:
model_uri = f"runs:/ed91e81d7ca644728f3e223fa3cfac92/model"
model = mlflow.pyfunc.load_model(model_uri)

#### Actualizar el modelo y tener una nueva version del Modelo de MlFlow Registry

In [0]:
import mlflow

# Definir el nombre del modelo y la nueva URI del modelo (para la versión 2)
model_name = "Example_rnn"
new_model_uri = 'runs:/3fdcef4bd83943ec9dc2188bc13045f4/model'  # Ejemplo de nueva URI del modelo

# Registrar la nueva versión del modelo con la misma nombre pero con una nueva URI
registered_model_version = mlflow.register_model(new_model_uri, model_name)

# Si deseas, puedes obtener el ID de la nueva versión del modelo registrado
new_version_id = registered_model_version.version
new_version_id

#### Validacion del Modelo antes de pasasr a Staging

In [0]:
#Obtener el modelo  in trancision, su nombre y version por el Webbook
#model_name, model_version = fech_webbook_data()
import mlflow
# Interactuar con el servidor de MLflow
client = mlflow.tracking.MlflowClient()  
model_name = "Example_rnn"

model_info = client.get_registered_model(model_name)
model_version = model_info.latest_versions[0].version #Ultima version del modelo en MlFlow Registry

models_details = client.get_model_version(model_name, model_version)
run_info = client.get_run(run_id=models_details.run_id)

In [0]:
run_info

#### Validacion de la prediccion

In [0]:
from databricks.feature_store import FeatureStoreClient

In [0]:
from databricks.feature_store import FeatureStoreClient

fs = FeatureStoreClient()

#Leer de Feature Store 
data_source = run_info.data.tags['db_table']
features = fs.read_table(data_source)

#Load model as a spark UDF
model_uri = f"models:/{model_name}/{model_version}"
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

#Selecciona las columnas de la tabla de caracteristicas, por el esquema input del modelo
input_columns_names = loaded_model.metadata.get_input_schema().input_names()

#Predict in spark dataframe
try:
    display(features.withColumn('predictions', loaded_model(*input_columns_names)))
    client.set_model_version_tag(name=model_name, version=model_version, key="predicts", value=1)
except Exception:
    print("No se puede predecir las caracteristicas,")
    client.set_model_version_tag(name=model_name, version=model_version, key="predicts", value=0)
    pass


In [0]:
import mlflow

# Nombre del modelo registrado
model_name = "MLops_kerastf_v3"

# Crear una instancia del cliente de MLflow
client = mlflow.tracking.MlflowClient()

# Obtener la información sobre el modelo
model_info = client.get_registered_model(model_name)

In [0]:
model_info

In [0]:
# Estado al que deseas cambiar el modelo (p. ej., "Production", "Staging", "Archived")
new_stage = "None"

# Versión específica que deseas cambiar de estado
#target_version = model_info.latest_versions[0].version # ultima version del modelo registrado en mlflow registry
target_version = 1


# Cambiar el estado del modelo
client.transition_model_version_stage(
    name=model_name,
    version = target_version,  #
    stage=new_stage
)


In [0]:
# Estado al que deseas cambiar el modelo (p. ej., "Production", "Staging", "Archived")
new_stage = "Production"

# Cambiar el estado del modelo
client.transition_model_version_stage(
    name=model_name,
    version=model_info.latest_versions[0].version,
    stage=new_stage
)

In [0]:
import matplotlib.pyplot as plt
# Crear gráfico de pérdida durante el entrenamiento
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
plt.plot(epochs, val_loss, 'r', label='Pérdida en validación')
plt.title('Pérdida durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

Modelo 2 : Redes Tradicionales

In [0]:
#Librerias para redes neuronales(LSTM)
from tensorflow.keras.layers import Dense, LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.optimizers import Adam
#from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# 3. Construccion del modelo de RED NEURONAL
# Aseguramos que los resultados sean "reproducibles" en cada ejecucion de tensorflow(pesos iniciales aleatorios)
np.random.seed(42)
tf.random.set_seed(42)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

#Arquitectura 2:
model_rnn = Sequential()
model_rnn.add(Dense(15, activation='relu', input_shape=[5]))
model_rnn.add(Dense(7, activation='relu'))
model_rnn.add(Dense(1, activation='linear'))

# Compilar el modelo
model_rnn.compile(loss='mean_squared_error', optimizer='adam')
# Imprimir un resumen del modelo
model_rnn.summary()

# Entrenar el modelo
# Red LSTM
#history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=5, validation_data=(X_test_rnn.reshape(-1, 1, 5),y_test_rnn), batch_size=5, verbose=1)
# Red Tradicional
history = model_rnn.fit(X_train_rnn, y_train_rnn, epochs=4, validation_data=(X_test_rnn,y_test_rnn), batch_size=5, verbose=1)

# 4. Validacion del Modelo con la metrica de RSME
# VALIDACION 1 del Modelo
# Obtener la pérdida en el conjunto de entrenamiento y el conjunto de validación
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Calcular el RMSE en el conjunto de entrenamiento y el conjunto de validación
train_rmse = np.sqrt(train_loss)
val_rmse = np.sqrt(val_loss)

# VALIDACION 2 del Modelo
# Hacer predicciones en el conjunto de prueba Red LSTM
#y_pred_rnn = model_rnn.predict(X_test_rnn.reshape(-1, 1, 5)) # (n_samples, n_pasos, n_variables)
# Hacer predicciones en el conjunto de prueba Red Tradicional
y_pred_rnn = model_rnn.predict(X_test_rnn) # (n_samples, n_pasos, n_variables)

# Redondear los valores de y_pred al entero más cercano
y_pred_rnn = np.round(y_pred_rnn).astype('int64')

# Calcular el error cuadrático medio en el conjunto de prueba
mse = mean_squared_error(y_test_rnn, y_pred_rnn)
# Calcular la raíz cuadrada del error cuadrático medio
rmse = np.sqrt(mse)
print(f'Raíz del Error Cuadrático Medio en el conjunto de prueba: {rmse}')

In [0]:
import matplotlib.pyplot as plt
# Crear gráfico de pérdida durante el entrenamiento
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
plt.plot(epochs, val_loss, 'r', label='Pérdida en validación')
plt.title('Pérdida durante el entrenamiento y la validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

In [0]:
np.unique(y_pred_rnn)

In [0]:
# Registrar un modelo en MLflow
model_path = "dbfs:/databricks/mlflow-tracking/8323b80e926241ee8763b7dc43085602/3fdcef4bd83943ec9dc2188bc13045f4/artifacts/model"
model_name = "MLops_kerastf_v1"
mlflow.register_model(model_path, model_name)

In [0]:
#ver el directorio local de databrinks mas no el local de .os
dbutils.fs.ls("dbfs:/")

### Guardar Modelo en Azure Storage

In [0]:
#1. Guardar en el directorio LOCAL (.OS)
model_rnn.save('/nuevomodelo/modelrnn_nuevo')

In [0]:
#2. Verificar que se ha guardado en el directorio LOCAL (.OS)
import os
os.listdir("/modelornn_version1_d4m")

Opcion 1: Cargar el Modelo ML directamente del Directorio (.OS)

In [0]:
# Cargar el modelo desde el contenedor de almacenamiento Azure
loaded_model_nuevo = tf.keras.models.load_model("/nuevomodelo/modelrnn_nuevo")

Opcion  2: Copiar el Modelo ML desde el Directorio LOCAL(.OS)  HASTA  Directorio de DBFS de Azure DataBrinks

In [0]:
#copie los datos desde LOCAL(.OS) a DBFS como dbfs (DIRECTORIO DE DATABRINKS)
dbutils.fs.cp("file:/nuevomodelo/modelrnn_nuevo", "dbfs:/nuevomodelo/modelrnn_nuevo", recurse=True) # True: si copias una carpeta, False: si #solo es archivo

Verificar que se ha migrado desde Directorio LOCAL(.OS)  Hacia Directorio DBFS

In [0]:
display(dbutils.fs.ls("/nuevomodelo/modelrnn_nuevo"))

Copiar el modelo desde el directorio DBFS al contenedor de almacenamiento Azure Storage

In [0]:
# Copiar el modelo desde el directorio DBFS al contenedor de almacenamiento Azure Storage
dbutils.fs.cp("/nuevomodelo/modelrnn_nuevo", "/mnt/datalakemlopsd4m/presentation/modelrnn_nuevo", recurse=True) #True:Copycarpeta,#False:Copyarchivo

In [0]:
import pandas as pd
import mlflow
import mlflow.keras
import mlflow.tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# # Establecer la URI de seguimiento de MLflow
# mlflow.set_tracking_uri("databricks")  # Usa "databricks" para Databricks

# # Crear un nuevo experimento si no existe
# experiment_name = "nombre_del_experimento"
# mlflow.set_experiment(experiment_name)

# Cargar los datos
df_delta = spark.read.format("delta").load("/mnt/datalakemlopsd4m/presentation/datmarcobre_fengineer_tablacaract_delta")
datos = df_delta.toPandas()

# Separar las variables independientes y dependiente
X = datos[['capacidad_en_volumen_equipo_carguio_m3',
           'capacidad_en_peso_equipo_carguio',
           'capacidad_en_peso_equipo_acarreo',
           'densidad_inicial_poligono_creado_tn/m3',
           'capacidad_en_volumen_equipo_acarreo_m3']].values
y = datos['numero_pases_carguio'].values

# Construir el modelo de red neuronal
np.random.seed(42)
tf.random.set_seed(42)

X_train_rnn, X_test_rnn, y_train_rnn, y_test_rnn = train_test_split(X, y, test_size=0.2, random_state=42)

mlflow.tensorflow.autolog()  # Keras y TensorFlow

# Entrenar el modelo
with mlflow.start_run(run_name='experimento_mlflow_keras_tf_3_21_24'):  #Iniciliza un nuevo experimento de MlFlow
    model_rnn = Sequential()
    model_rnn.add(LSTM(30, activation='relu', input_shape=(1, 5)))
    model_rnn.add(Dense(60, activation='relu'))
    model_rnn.add(Dense(30, activation='relu'))
    model_rnn.add(Dense(15, activation='relu'))
    model_rnn.add(Dense(1, activation='linear'))

    model_rnn.compile(loss='mean_squared_error', optimizer='adam')

    history = model_rnn.fit(X_train_rnn.reshape(-1, 1, 5), y_train_rnn, epochs=1, 
                            validation_data=(X_test_rnn.reshape(-1, 1, 5), y_test_rnn), 
                            batch_size=5, verbose=1)

    # Registrar los parámetros del modelo
    mlflow.log_params({
        "input_shape": (1, 5),
        "lstm_units": 30,
        "dense_layers": [60, 30, 15],
        "activation": "relu",
        "optimizer": "adam",
        "loss_function": "mean_squared_error",
        "epochs": 1,
        "batch_size": 5
    })

    # Registrar la estructura del modelo en MLflow
    mlflow.keras.log_model(model_rnn, "keras_tf_3_21_24")

    # Calcular métricas
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']
    train_rmse = np.sqrt(train_loss)
    test_rmse = np.sqrt(test_loss)

    # Registrar métricas de RMSE
    mlflow.log_metric("train_RMSE", train_rmse[-1])
    mlflow.log_metric("test_RMSE", test_rmse[-1])

    # Crear gráfico de pérdida durante el entrenamiento
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'bo', label='Pérdida en entrenamiento')
    plt.plot(epochs, test_loss, 'r', label='Pérdida en validación')
    plt.title('Pérdida durante el entrenamiento y la validación')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()
    plt.savefig("kerasplot_3_21_24.png")
    mlflow.log_artifact("kerasplot_3_21_24.png")

    # Obtener la ruta del modelo registrado
    model_path = mlflow.get_artifact_uri("keras_tf_3_21_24")

# Registrar el modelo en MLflow
model_name = "MLops_kerastf_3_21_24"
mlflow.register_model(model_path, model_name)